In [ ]:
#%pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-huggingface-api -U -q

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file

open_api_key = os.getenv('OPENAI_API_KEY')

In [2]:


from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b



In [3]:
from llama_index.llms.openai import OpenAI


# Initialize the model with the required parameters
openai_model_name =  "gpt-4o-mini"
model = OpenAI(model=openai_model_name, api_key=open_api_key)

agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=model,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

In [4]:
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp


Called tool:  add {'a': 2, 'b': 2} => 4

Called tool:  multiply {'a': 4, 'b': 2} => 8
The result of \((2 + 2) * 2\) is \(8\).

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The result of \\((2 + 2) * 2\\) is \\(8\\).')]), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='call_FpsIS2DZO2xxkpIzGhYBRGAQ', tool_output=ToolOutput(content='4', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 2}, tool_id='call_D8BbkAm65tQvFsocCzKslQa0', tool_output=ToolOutput(content='8', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 4, 'b': 2}}, raw_output=8, is_error=False), return_direct=False)], raw={'id': 'chatcmpl-BBKzZi4HI6bPbgAynzD62ql4w3pZG', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1742042001, 'model': 'gpt-4o-

In [5]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')]), tool_calls=[], raw={'id': 'chatcmpl-BBKzahHKeosJeWiLXbSeve60aqi3U', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1742042002, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', 'service_tier': 'default', 'system_fingerprint': 'fp_3267753c5d', 'usage': None}, current_agent_name='Agent')

In [8]:
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a query engine

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [11]:
from llama_index.core.tools import QueryEngineTool

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=model)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=model,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)


In [ ]:
# works Jupyter notebook only, but not py export
'''
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp
'''


Called tool:  personas {'input': 'science fiction'} => Empty Response
It seems that there are no persona descriptions available in the database for "science fiction." If you have another topic or specific persona in mind, please let me know!

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='It seems that there are no persona descriptions available in the database for "science fiction." If you have another topic or specific persona in mind, please let me know!')]), tool_calls=[ToolCallResult(tool_name='personas', tool_kwargs={'input': 'science fiction'}, tool_id='call_WpGB82KXXo4emLfnFHLUVrXr', tool_output=ToolOutput(content='Empty Response', tool_name='personas', raw_input={'input': 'science fiction'}, raw_output=Response(response='Empty Response', source_nodes=[], metadata=None), is_error=False), return_direct=False)], raw={'id': 'chatcmpl-BBL5aMOhZmrnfmd79LzKjzbh8nYaT', 'choices': [{'delta': {'content': None, 'function_call': None, 'refusal': None, 'role': None, 'tool_calls': None}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'created': 1742042374, 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion.chunk', '

In [13]:
async def main():
    # Run first agent query
    handler = agent.run("What is (2 + 2) * 2?")
    async for ev in handler.stream_events():
        if isinstance(ev, ToolCallResult):
            print("\nCalled tool:", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
        elif isinstance(ev, AgentStream):  # showing the thought process
            print(ev.delta, end="", flush=True)
    resp = await handler
    print("\nFinal response:", resp)
    
    # Run conversation using context
    ctx = Context(agent)
    response = await agent.run("My name is Bob.", ctx=ctx)
    response = await agent.run("What was my name again?", ctx=ctx)
    print("\nConversation response:", response)
    
    # Run query engine agent query
    handler = query_engine_agent.run(
        "Search the database for 'science fiction' and return some persona descriptions."
    )
    async for ev in handler.stream_events():
        if isinstance(ev, ToolCallResult):
            print("\nCalled tool:", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
        elif isinstance(ev, AgentStream):  # showing the thought process
            print(ev.delta, end="", flush=True)
    resp = await handler
    print("\nQuery engine final response:", resp)

In [16]:
if __name__ == "__main__":
    asyncio.run(main())

NameError: name 'asyncio' is not defined